Setup and create tables

In [2]:
import sqlite3
import pandas as pd

# Connect to (or create) the database
conn = sqlite3.connect('student_grades.db')
cursor = conn.cursor()

# Create Students table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Students (
    StudentID INTEGER PRIMARY KEY AUTOINCREMENT,
    StudentName TEXT NOT NULL,
    Age INTEGER,
    GradeLevel TEXT
)
''')

# Create Subjects table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Subjects (
    SubjectID INTEGER PRIMARY KEY AUTOINCREMENT,
    SubjectName TEXT NOT NULL
)
''')

# Create Grades table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Grades (
    GradeID INTEGER PRIMARY KEY AUTOINCREMENT,
    StudentID INTEGER,
    SubjectID INTEGER,
    Score REAL,
    FOREIGN KEY(StudentID) REFERENCES Students(StudentID),
    FOREIGN KEY(SubjectID) REFERENCES Subjects(SubjectID)
)
''')

conn.commit()
print("Tables created successfully!")


Tables created successfully!


Insert sample data

In [3]:
# Insert students
students = [('Alice', 14, '9th'), ('Bob', 15, '10th'), ('Charlie', 14, '9th')]
cursor.executemany('INSERT INTO Students (StudentName, Age, GradeLevel) VALUES (?, ?, ?)', students)

# Insert subjects
subjects = [('Math',), ('English',), ('Science',)]
cursor.executemany('INSERT INTO Subjects (SubjectName) VALUES (?)', subjects)

# Insert grades (StudentID, SubjectID, Score)
grades = [
    (1, 1, 85.5),
    (1, 2, 90.0),
    (1, 3, 88.0),
    (2, 1, 78.0),
    (2, 2, 82.5),
    (2, 3, 79.0),
    (3, 1, 92.0),
    (3, 2, 88.5),
    (3, 3, 91.0)
]
cursor.executemany('INSERT INTO Grades (StudentID, SubjectID, Score) VALUES (?, ?, ?)', grades)

conn.commit()
print("Sample data inserted!")


Sample data inserted!


Query average score per student


In [6]:
query_avg_student = '''
SELECT s.StudentName, AVG(g.Score) AS AverageScore
FROM Grades g
JOIN Students s ON g.StudentID = s.StudentID
GROUP BY s.StudentName
ORDER BY AverageScore DESC
'''

df_avg_student = pd.read_sql_query(query_avg_student, conn)
print("Average Scores per Student:")
df_avg_student


Average Scores per Student:


,StudentName,AverageScore
0,Charlie,90.500000
1,Alice,87.833333
2,Bob,79.833333


Query average score per subject

In [9]:
query_avg_subject = '''
SELECT sub.SubjectName, AVG(g.Score) AS AverageScore
FROM Grades g
JOIN Subjects sub ON g.SubjectID = sub.SubjectID
GROUP BY sub.SubjectName
ORDER BY AverageScore DESC
'''

df_avg_subject = pd.read_sql_query(query_avg_subject, conn)
print("Average Scores per Subject:")
df_avg_subject


Average Scores per Subject:


,SubjectName,AverageScore
0,English,87.000000
1,Science,86.000000
2,Math,85.166667


Query top performing students (average > 85)

In [10]:
query_top_students = '''
SELECT s.StudentName, AVG(g.Score) AS AverageScore
FROM Grades g
JOIN Students s ON g.StudentID = s.StudentID
GROUP BY s.StudentName
HAVING AverageScore > 85
ORDER BY AverageScore DESC
'''

df_top_students = pd.read_sql_query(query_top_students, conn)
print("Top Performing Students (Average > 85):")
df_top_students


Top Performing Students (Average > 85):


,StudentName,AverageScore
0,Charlie,90.500000
1,Alice,87.833333
